# Example Gallery

This notebook collects some examples of what you can do with the crate.

Until some more focused and didactic tutorial is written, this is the main practical documentation.

For a more in-depth overview and discussion of the concepts, check out the blog posts linked in the repository.

In [ ]:
// build and load the crate
:dep plotters = { version = "^0.3.7", features = ["evcxr", "all_series"] }
:dep tilezz = { path = ".." }

// import everything we need (for convenience)
use tilezz::cyclotomic::*;
use tilezz::snake::{Snake, Turtle};
use tilezz::rat::Rat;

use plotters::prelude::*;
use tilezz::plotters::{plot_tile, tile_chart, TileStyle};

## Minimal example

Let us describe and render a tile.

In [ ]:
evcxr_figure((500,500), |root| {
    let _ = root.fill(&WHITE);
    let root = root.margin(10, 10, 10, 10);
    // ----
    
    let seq_zz24: Vec<i8> = vec![6, 4, 0, 4, -6, 4, 6, 4, -6, 4, 6, -4, 6, -4];
    let s: Snake<ZZ12> = spectre();

    // ----
    let p = s.to_polyline_f64(Turtle::default());
    plot_tile(&root, &p, &TileStyle::default());
    Ok(())
})

Now you might recognize this as the legendary spectre tile. You see the vertices numbered in counterclockwise.

The sequence of numbers we have entered describes the **external angle** of the current polygon vertex. We work in $Z[\zeta_{12}]$, so a full rotation is $12$, a half-rotation is 6. **Angles describing a polygon must have an absolute value less than half a rotation**, i.e. here between $-6$ and $6$.

Let us rotate the polygon by one step. Now we'll use the utility function to give us the angle sequence of the spectre, but we will use the `Rat` type instead of a `Snake`.

In [ ]:
evcxr_figure((500,500), |root| {
    let _ = root.fill(&WHITE);
    let root = root.margin(10, 10, 10, 10);
    // ----
    
    let mut r: Rat<ZZ12> = Rat::try_from(&spectre()).unwrap().cycle(1);  // <-- look here

    // ----
    let p = r.to_polyline_f64(Turtle::default());
    plot_tile(&root, &p, &TileStyle::default());
    Ok(())
})

You can see that the first vertex is the next in counter-clockwise direction compared to the previous image, but the tile is rotated very differently in the plot.

The reason is that the first vertex is assumed to be at the origin and the first external angle is between the real line and the first line segment.

To rotate the tile in the plot, we have to specify the orientation of the `Turtle`.

In [ ]:
evcxr_figure((500,500), |root| {
    let _ = root.fill(&WHITE);
    let root = root.margin(10, 10, 10, 10);
    // ----
    
    let mut r: Rat<ZZ12> = Rat::try_from(&spectre()).unwrap().cycle(1);

    // ----
    let p = r.to_polyline_f64(Turtle::new(0.into(), 3)); // <-- look here
    plot_tile(&root, &p, &TileStyle::default());
    Ok(())
})

Fixme: helper function to hide boilerplate
split out basic tutorial for using rats, snakes and turtles and explaining how the geometry maps to the plot.

In [ ]:
// In this cell, let us look at the customization options for the plotting.

// Build a customized tile style.
fn my_custom_style<'a>() -> TileStyle<'a> {
    let mut st = TileStyle::default();
    st.label_font = st.label_font.color(&RED);
    st.border_style = st.border_style.stroke_width(5);
    st.fill_style.color = TRANSPARENT;
    st.node_zero_only = true;
    st.node_size = 10;
    st.node_style = BLUE.filled();
    st.node_labels = false;
    return st;
}

evcxr_figure((2000,1000), |root| {
    let _ = root.fill(&WHITE);
    let root = root.margin(10, 10, 10, 10);

    // --------
    // Let us glue two spectre tiles together to get a "Mystic"!

    let spectre: Rat<ZZ12> = Rat::try_from(&spectre()).unwrap();
    // glue a copy of the spectre to itself to get a mystic
    let mystic = spectre.glue((2, 0), &spectre);
    // take the mirror image and shift the starting point
    let spectre_mod = spectre.reflected().cycle(5);

    // --------
    // Now let's plot this to see what we have done!

    // get concrete points and define styles for the tiles
    let t1 = spectre.to_polyline_f64(Turtle::new(0.into(), -2));
    let s1 = TileStyle::default().with_label("The Spectre");

    let t2 = mystic.to_polyline_f64(Turtle::new(ZZ12::from((2, 3)), -3));
    let s2 = TileStyle::default().with_label("The Mystic");

    let t3 = spectre_mod.to_polyline_f64(Turtle::new(0.into(), -1));
    let s3 = my_custom_style().with_label("Customized Spectre");
    let (left, right) = root.split_horizontally(1000);

    // plot two tiles in the chart on the left
    let tiles = vec![(t1.as_slice(), &s1), (t2.as_slice(), &s2)];
    let (mut c1, plot_tiles) = tile_chart(
        &mut ChartBuilder::on(&left)
            .caption("My Custom Tile Plot", ("sans-serif", 40).into_font())
            .x_label_area_size(20)
            .y_label_area_size(40),
        tiles.as_slice(),
    );
    c1.configure_mesh().draw().unwrap();
    plot_tiles(&mut c1);

    // plot a tile in the chart on the right
    right.fill(&GREEN.mix(0.1)).unwrap();
    plot_tile(&right, t3.as_slice(), &s3);
    
    Ok(())
})

In [ ]:
evcxr_figure((500,500), |root| {
    let _ = root.fill(&WHITE);
    let root = root.margin(10, 10, 10, 10);

    // In this example, let's build two simple tiles and glue them together,
    // but plot them into the same coordinate system.

    // As the abstract polygon representations (i.e. rats) are abstracted from coordinates,
    // nothing changes at this level. Coordinates do not exist in this perspective.
    let a = Rat::<ZZ12>::try_from(&[0, 0, 3, 0, 3, 3, -3, -3, 3, 3, 0, 3]).unwrap();
    let b = Rat::<ZZ12>::try_from(&[0, 0, 3, 3, 0, 0, 3, 3]).unwrap();
    let c = a.glue_unchecked((0, 3), &b);

    // By default, all polygons start and end at the origin.
    // We determine the placement of a polygon by defining a different turtle,
    // i.e. starting point for tracing out the vertices in the plane.
    let a_pts = a.to_polyline_f64(Turtle::default());
    let b_pts = b.to_polyline_f64(Turtle::new(ZZ12::from((0, 3)), 0));
    let c_pts = c.to_polyline_f64(Turtle::new(ZZ12::from((5, 3)), 3));

    let s = TileStyle::default().with_fill(TRANSPARENT.into());
    let (s_a, s_b, s_c) = (s.clone().with_label("a"), s.clone().with_label("b"), s.clone().with_label("c"));
    let tiles2 = vec![
        (a_pts.as_slice(), &s_a),
        (b_pts.as_slice(), &s_b),
        (c_pts.as_slice(), &s_c),
    ];
    let (mut c, plot_tiles) = tile_chart(&mut ChartBuilder::on(&root), tiles2.as_slice());
    plot_tiles(&mut c);

    Ok(())
})    

In [ ]:
evcxr_figure((500,500), |root| {
    let _ = root.fill(&WHITE);
    let root = root.margin(10, 10, 10, 10);

    let unchecked = Snake::<ZZ12>::from_slice_unchecked(&[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]);
    let pts = unchecked.to_polyline_f64(Turtle::default());
    plot_tile(
        &root,
        pts.as_slice(),
        &TileStyle::default().with_fill(TRANSPARENT.into()),
    );
    Ok(())
})